<a href="https://colab.research.google.com/github/priyvantkumar9661-png/Ai-story-teller/blob/main/Ai_story_teller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%env GEMINI_API_KEY=**********************************

In [ ]:
!pip install -q transformers pillow google-generativeai

In [ ]:
from google import genai
import os
client=genai.Client()

In [ ]:
if "GEMINI_API_KEY" not in os.environ:
  print("Please set your Gemini API key in the environment variable GEMINI_API_KEY")
else:
  client=genai.Client()
  Model="gemini-2.5-flash"

In [ ]:
prompt=input("Enter your story prompt and press enter:\n")
if prompt.strip()=="":
  print("No prompt entered, Existing.")
else:
  print(f"Generating story for prompt: {prompt}")
  print("It may take few seconds.")
  try:
    resp=client.models.generate_content(model=Model, contents=[prompt])
    print ("\n----Generated Story----\n")
    print(resp.text)
  except Exception as e:
    print(f"Error occured while generating story : {e}")

In [ ]:
!pip install -q transformers pillow google-generativeai timm

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import os
import io

In [ ]:
if "GEMINI_API_KEY" not in os.environ:
  print("Please set your Gemini API key in the environment variable GEMINI_API_KEY")
else:
  client=genai.Client()
  Model="gemini-2.5-flash"

In [ ]:
processor=BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model=BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

In [ ]:
from google.colab import files
uploaded=files.upload()

for fn in uploaded.keys():
  image=Image.open(fn).convert('RGB')
  display(image)

In [ ]:
=processor(images=image, return_tensors='pt')
out=model.generate(**inputs)

caption=processor.decode(out[0], skip_special_tokens=True)
print(f"Caption generated by Blip:\n {caption}")

In [ ]:
story_prompt=(f"Write a short story (around 500-700 words) based on this scene description:\n {caption}")
print(story_prompt)

print("Sending this to Gemini. \n")

response=client.models.generate_content(model=Model, contents={story_prompt})
story=response.text
print("\n----Generated Story----\n", story)

In [ ]:
with open ("generated_story.txt","w")as f:
  f.write(story)

from google.colab import files
files.download("generated_story.txt")

In [ ]:
!pip install -q ipywidgets

In [ ]:
from google.colab import files
from PIL import Image
import io

uploaded=files.upload()

images=[]
image_names=[]
for name,file in uploaded.items():
  image=Image.open(io.BytesIO(file)).convert('RGB')
  image_names.append(name)
  display(image)

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration

processor=BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
blip_model=BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

captions=[]

for img in images:
  inputs=processor(images=img, return_tensors='pt')
  out=blip_model.generate(**inputs, max_new_tokens=30)
  caption=processor.decode(out[0],skip_special_token=True)
  captions.append(caption)

print("Captions generated from images:")
for i,caption in enumerate(captions):
  print(f"{image_names[i]}: {caption}")

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

tone_dropdown= widgets.Dropdown(
    options=["whimsical", "adventurous","suspenseful","romantic","Sci-fi","mystery","Funny","Knowledgeable"],
    value="Sci-fi",
    description="Tone:"
)

length_dropdown=widgets.Dropdown(
    options=["Short(100-200 words)","Medium(300-500 words)","Long(600-800 words)"],
    value="Medium(300-500 words)",
    description="Length:"
)

generate_button=widgets.Button(description="Generate Story")
output_box=widgets.Output()

display(tone_dropdown, length_dropdown, output_box,generate_button)

In [ ]:
def on_generate_clicked(b):
  with output_box:
    clear_output()

    tone=tone_dropdown.value
    length_map={
        "Short(100-200 words)":"100-200 words",
        "Medium(300-500 words)":"300-500 words",
        "Long(600-800 words)":"600-800 words"
    }
    length=length_map[length_dropdown.value]

    caption_prompt="\n".join([f"-{c}" for c in captions])

    outline_prompt=(
        f"Using the following scene description, create a 4-chapter story outline."
        f"Each chapter should have a title and a short summary.\n\n"
        f"{caption_prompt}\n\n Outline:"
    )

    try:
      outline_response=client.models.generate_content(model=Model, contents=outline_prompt)
      outline_text=outline_response.text
      print("🧾 Story Outline:\n")
      print(outline_text)

      full_story=""
      for i in range(1,4):
        change_prompt=(
            f"Using the outline below, write Chapter {i} in a {tone} tone."
            f"Make it {length}. Add vivid details, good pacing, and consistent characters.\n\n"
            f"{outline_text}\n\nChapter {i}:"
        )

        chapter_response=client.models.generate_content(model=Model, contents=change_prompt)
        chapter_text=chapter_response.text
        print(f"\n\nChapter {i}:\n")
        print(chapter_text)
        full_story+= f"\n\nChapter {i}:\n{chapter_text}"

      with open("multi_image_story.txt","w") as f:
        f.write(full_story)
      print("\n Story saved as multi_image_story.txt")
      files.download("multi_image_story.txt")
    except Exception as e:
      print(f"Error occured while generating story: {e}")

generate_button.on_click(on_generate_clicked)

In [ ]:
!pip install -q gtts reportlab

In [ ]:
#You can paste your story here or load from file
story_text="""
Chapter 1:

The ancient lighthouse, a skeletal finger against the perpetually bruised sky, beckoned Elara from the treacherous, fog-shrouded shore of Serpent's Tooth Island. She’d sought oblivion here, a desolate crag gnawing at the edge of the charted world, hoping its oppressive silence would finally drown the spectral cries of the *Argus*, her last failed expedition. But the lighthouse, a lonely sentinel with its once brilliant eye long extinguished, offered not peace, but a colder, more potent allure – the promise of forgotten lore, of answers perhaps to questions she hadn’t dared to voice.

Salt-laced wind, sharp as shattered glass, tore at her waterproof anorak as she approached the crumbling structure. The beacon’s glass was long gone, its lantern room a crown of jagged teeth biting into the grey heavens. Rust bled down the ironwork like ancient tears, staining the concrete base. The air itself felt heavy, thick with the tang of decay and something else – a deep, almost resonant stillness that pressed in on her, urging caution, yet pulling her forward with an irresistible gravity.

The heavy, iron-bound door groaned a protest as she forced it inward, releasing a puff of stagnant air. Inside, the chill was profound, a sepulchral cold that seeped into her bones. The spiral staircase, a helix of wrought iron and timeworn stone, rose into the gloom, its steps groaning under the phantom weight of forgotten keepers. Dust motes, ancient and luminous, danced in the faint, weak light filtering through grime-caked windows, each particle a tiny, suspended galaxy. Her boots echoed loudly, the only sound in the lighthouse's vast, eerie silence, each step a deliberate intrusion into a place that felt more tomb than tower.

Higher and higher she climbed, the ascent a slow, grinding battle against time and the building's lingering ghosts. The oppressive atmosphere intensified, pressing down, making each breath a conscious effort. Finally, after what felt like an eternity, she emerged into the shattered lantern room. The wind howled through the gaping wounds where glass once stood, carrying with it the perpetual stench of salt and decay. Rusted machinery, monstrous and indecipherable, lay dormant, twisted metal relics of a bygone era, perhaps even a different civilization.

Amidst this silent tableau of forgotten purpose, nestled within a coil of corroded copper cable, lay a single, leather-bound journal. Its cover, dark and cracked, bore an ornate, forgotten symbol – a swirling vortex perhaps, or an alien constellation. The pages, brittle with age, seemed to hum with a faint, unheard energy. Elara’s gloved fingers trembled as she reached for it, the weight of the island’s silence and the lighthouse’s profound mystery settling around her like a shroud. This was not the solace she’d sought, but something far more unsettling, far more compelling. The sentinel had been silent for too long, and its secrets were now poised to speak.


Chapter 2:

Elara found temporary refuge in the lee of a wind-battered outcrop, the chilling sea mist a constant, silent presence. The ancient journal felt unnervingly light in her hands, its leather cover, once rich and supple, now cracked like sun-baked earth. The ornate symbol, a fractal pattern of intertwining waves and stars, seemed to shimmer faintly under the diffuse, grey light, a ghost of an unknown energy. With a deep breath, she carefully opened its brittle pages, the dry scent of aged paper and forgotten secrets filling her nostrils.

The script, a spidery but precise hand, belonged to Silas Croft, the lighthouse's last keeper. His initial entries were a chronicle of solitude: meticulous weather observations, the rhythmic clang of the bell buoy, the endless vigil against the encroaching sea. Elara, herself familiar with the quiet companionship of solitude, felt a kinship with the unseen man. But soon, the entries began to subtly shift, a creeping unease seeping into the precise penmanship.

"Day 187," one entry read, "The ocean’s pulse falters. Tides recede further, exposing skeletal fingers of reef never before seen. Then, they surge, a liquid wall, without the moon’s command." Later, Silas documented inexplicable atmospheric phenomena: auroras bleeding across the twilight sky in the wrong season, the air pressure dropping to dangerous lows without a single cloud, strange luminous jellyfish ascending from the deep, pulsing with a cold, internal light.

His focus narrowed, obsessive. He spoke of whispers, "a frequency beneath the waves," and then, explicitly, of an "ancient mechanism." It lay, he posited, deep beneath Serpent's Tooth, a relic of a civilization lost to the abyssal trenches, designed to command the very hydrographic and atmospheric currents of the region. Silas believed these strange shifts were not natural aberrations, but the initial tremors of this forgotten apparatus stirring, its primeval gears grinding back to life.

As Elara turned the pages, Silas’s handwriting grew more erratic, his theories more frantic. "They are waking it," he scribbled, his lines slanting wildly. "The cosmic dark, stirring in the abyss, seeks to reclaim its grasp on the currents, to unravel the world's carefully woven climate. This island, this lighthouse, is a mere scar upon the surface, but beneath… beneath lies the anchor, the heart." He detailed a "cosmic threat," something vast and ancient, its awakening tied to the reanimation of the hidden mechanism. Was this the desperate ranting of a man consumed by isolation, or a chilling, forgotten warning echoing through time? Elara’s scientific mind warred with a growing, profound dread. The symbol on the cover seemed to throb, an intricate key to a terrifying, untold truth.


Chapter 3:

The sky didn’t just darken; it curdled. A furious shriek of wind tore across Serpent’s Tooth Island, not merely gusting but carrying a palpable weight of cosmic intent. Rain lashed against the grimy windows of Elara’s temporary shelter, each drop a tiny hammer-blow, echoing the furious, desperate scrawl of Silas Croft’s final journal entries. *“The tides… they answer the call. The sky bleeds… and the mechanism awakens.”*

The words, once dismissed as the ramblings of a fractured mind, now manifested outside with terrifying clarity. This wasn’t a natural storm; it was an apocalyptic symphony of wind and water, precisely as Silas had described. A cold dread, starker than the whipping rain, settled in Elara’s gut. His warnings weren't just real; they were unfolding around her.

Driven by a surge of desperate resolve, Elara snatched up the journal. Deep within its brittle pages, hidden beneath a faded drawing of the island’s profile, she found it: a cryptic etching, almost invisible against the aged parchment. It wasn't a map in the conventional sense, but a series of interconnected symbols, charting a path deep beneath the lighthouse foundation. Her cartographer’s eye quickly deciphered the subtle shifts in iconography, recognizing points of entry and ancient safeguards.

Battling the tempest, Elara returned to the groaning sentinel. The lighthouse, once a silent monument, was now a roaring conduit for the storm, its shattered lantern room howling like a banshee. Below the spiral staircase, where Silas’s final diagram had indicated, a concealed passage yawned open. The air grew thick with mineral tang and something else – a faint, almost imperceptible hum that resonated in her bones.

She descended into the earth, guided by the journal’s crude illumination. The passage was a labyrinth of worked stone, slick with brine, echoing with the deep thrum of the ocean. Pressure plates hummed with latent energy underfoot, ancient mechanisms demanding precise sequences. Tidal pools rose and fell with unnerving precision, some revealing hidden doorways, others guarding perilous drops into abyssal darkness. Luminescent fungi pulsed like watchful eyes in the gloom, casting an alien glow on the walls, etched with symbols that mirrored the journal’s ornate cover.

Finally, deep within the island's core, the passage opened into a vast, subterranean cavern. Before her stood a colossal, alien engine, dormant yet radiating immense, silent power. It wasn't built; it appeared to have grown from the very rock, a latticework of crystalline conduits and obsidian plating, pulsing with a dim, internal light that made the air crackle with ozone. This was it: the tide-controlling mechanism Silas had spoken of. Outside, the storm intensified, the very ground trembling with its fury. Elara understood then, with a crushing wave of clarity: Silas hadn't been trying to *control* this power; he’d been desperately trying to *contain* it. Now, it was awakening, poised to unleash chaos upon the sea and the unsuspecting coastal towns beyond.
"""

In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

def export_pdf(text, filename="story.pdf"):
  c=canvas.Canvas(filename, pagesize=letter)
  width,height=letter
  text_object=c.beginText(40,height-40)
  text_object.setFont("Helvetica", 12)

  for line in text.split("\n"):
    for subline in[line[i:i+90]for i in range(0, len(line),90)]:
      text_object.textLine(subline)
  c.drawText(text_object)
  c.showPage()
  c.save()

export_pdf(story_text)

from google.colab import files
files.download("story.pdf")

In [ ]:
from gtts import gTTS
from IPython.display import Audio, display
from google.colab import files

voices={
    "Deafult English (US Female)": {"lang":"en","tld":"com"},
    "British Accent":{"lang":"en","tld":"co.uk"},
    "Australian Accent":{"lang":"en","tld":"com.au"},
    "Indian Accent":{"lang":"en","tld":"co.in"},
    "Slow Reading Voice":{"lang":"en","tld":"com","slow":True},
    "Indian Accent":{"lang":"hi","tld":"co.in"}
}

for label,options in voices.items():
  print(f"Generating Audio: {label}")

  tts=gTTS(
      text=story_text,
      lang=options["lang"],
      tld=options.get("tld","com"),
      slow=options.get("slow",False)
  )

  filename = f"{label.replace(' ','_').lower()}.mp3"

  tts.save(filename)

  display(Audio(filename=filename,autoplay=False))

  files.download(filename)

In [ ]:
%%writefile app_streamlit_story.py
import streamlit as st # web app framework
from PIL import Image
import io, requests, os
import textwrap
from gtts import gTTS
from transformers import BlipProcessor, BlipForConditionalGeneration
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.utils import ImageReader
from pyngrok import ngrok #generate public link
import tempfile
import google.generativeai as genai
import torch

#Authentication
NGROK_AUTH_TOKEN = "*******************" #Use your own unique token
BACKGROUND_IMAGE_URL="https://i.postimg.cc/MGCVjtZ4/download.webp"
GEMINI_API_KEY="*****************************"

#Streamlit Page Setup/Style
st.set_page_config(page_title="Story Teller", layout="wide")

st.markdown(
    f"""
    <style>
    .stApp {{
        background-image: url("{BACKGROUND_IMAGE_URL}");
        background-size: cover;
        background-attachment: fixed;
    }}
    section[data-testid="stSidebar"] {{
        background: rgba(0,0,0,0.3);
        backdrop-filter: blur(10px);
        border-radius: 12px;
        padding: 10px;
    }}
    div[data-testid="stFileUploader"] {{
        background: rgba(255,255,255,0.2);
        border-radius: 10px;
        padding: 10px;
    }}
    html, body, h1, h2, h3, h4, h5, h6, p, div, span, label, li, input, textarea {{
        color: #93A8AC !important;
    }}
    .stButton>button, .stDownloadButton>button {{
        color: #93A8AC !important;
        border-color: #93A8AC;
    }}
    </style>
    """,
    unsafe_allow_html=True
)

st.title("Multi-Image Ai Story Teller")
st.markdown("Upload images -> Generate story -> Export as PDF & MP3")

with st.sidebar:
    tone = st.selectbox("Tone", ["Adventurous", "Whimsical", "Romantic", "Mysterious", "Humorous", "Calm"])
    length_label = st.selectbox("Length", ["Short (200-300 words)", "Medium (300-600 words)", "Long (600-1000 words)"])
    start_ngrok = st.checkbox("Start ngrok tunnel")
    if start_ngrok:
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        url = ngrok.connect(8501)
        st.success(f"Public URL: {url}")

uploaded_images = st.file_uploader("Upload multiple images", type=["jpg","jpeg","png"], accept_multiple_files=True)

#Caption model
@st.cache_resource
def load_models():
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda" if torch.cuda.is_available() else "cpu")
    return processor, model

processor, blip_model = load_models()

#config gemini
genai.configure(api_key=GEMINI_API_KEY)

@st.cache_resource
def load_gemini_model():
    return genai.GenerativeModel(model_name="models/gemini-2.5-flash")

gemini_model = load_gemini_model()

#captioning the images
def get_captions(images):
    captions = []
    for img in images:
        if img.mode != "RGB":
            img = img.convert("RGB")
        inputs = processor(images=img, return_tensors="pt").to(blip_model.device)
        out = blip_model.generate(**inputs)
        caption = processor.decode(out[0], skip_special_tokens=True)
        captions.append(caption)
    return captions

#Generaring the story
def generate_story(captions, tone, length_label):
    length_map= {
        "Short (200-300 words)": (200, 300, 800),
        "Medium (300-600 words)": (300, 600, 1200),
        "Long (600-1000 words)": (600, 1000, 1600)
    }
    min_words, max_words, max_tokens = length_map.get(length_label, (300, 600, 1200))

    prompt = (
    f"You are a creative writer. Write a {tone.lower()} story based on the following image captions:\n\n"
    + "\n".join([f"- {cap}" for cap in captions])
    + f"\n\nThe story should be vivid, engaging, and emotionally rich, with a coherent beginning, middle, and end."
    + f"\nMake it approximately between {min_words} and {max_words} words long."
)


    try:
        response = gemini_model.generate_content(
            contents=prompt,
            generation_config=genai.GenerationConfig(
                temperature=0.9,
                top_p=0.95,
                max_output_tokens=max_tokens
            )
        )
        return response.text.strip()
    except Exception as e:
        return f"❌ Error in generating story: {e}"

#Pdf generation
def create_pdf(story_text, images):
    buffer = io.BytesIO()
    c = canvas.Canvas(buffer, pagesize=A4)
    w, h = A4

    try:
        bg_img = Image.open(requests.get(BACKGROUND_IMAGE_URL, stream=True).raw).convert("RGB")
        bg = ImageReader(bg_img)
        c.drawImage(bg, 0, 0, width=w, height=h)
    except:
        pass

    c.setFont("Helvetica-Bold", 16)
    c.drawString(50, h - 50, "Generated Story")

    text = textwrap.wrap(story_text, 100)
    y = h - 80
    for line in text:
        if y < 80:
            c.showPage()
            y = h - 80
        c.drawString(50, y, line)
        y -= 15

    if images:
        c.showPage()
        c.setFont("Helvetica-Bold", 16)
        c.drawString(50, h - 50, "Uploaded Images")
        x, y = 50, h - 150
        for img in images:
            img.thumbnail((200, 200))
            c.drawImage(ImageReader(img), x, y, width=img.width, height=img.height)
            x += 220
            if x > w - 200:
                x = 50
                y -= 220
    c.save()
    buffer.seek(0)
    return buffer

#Audio generation
def create_audio(story):
    audio_bytes = io.BytesIO()
    tts = gTTS(story)
    tts.write_to_fp(audio_bytes)
    audio_bytes.seek(0)
    return audio_bytes

#Processing part
if st.button("Generate Story") and uploaded_images:
    pil_images = [Image.open(img) for img in uploaded_images]
    with st.spinner("Generating captions..."):
        captions = get_captions(pil_images)
        for i, cap in enumerate(captions):
            st.write(f"**Image {i+1}**: {cap}")

    with st.spinner("Generating story..."):
        story = generate_story(captions, tone, length_label)
        st.success("Story generated!")
        st.write(story)

    with st.spinner("Creating PDF..."):
        pdf_file = create_pdf(story, pil_images)
        st.download_button("📄 Download Story as PDF", data=pdf_file, file_name="story.pdf", mime="application/pdf")

    with st.spinner("Creating Audio..."):
        audio = create_audio(story)
        st.audio(audio)
        st.download_button("🔊 Download Story as MP3", data=audio, file_name="story.mp3", mime="audio/mpeg")

elif not uploaded_images:
    st.warning("Upload at least one image to begin.")

In [ ]:
!pip install -q streamlit pyngrok transformers torch gtts reportlab Pillow
!streamlit run app_streamlit_story.py --server.port 8501 &>/content/log.txt &

from pyngrok import ngrok
ngrok.set_auth_token("**************************") #Use your own unique token.
url = ngrok.connect(8501)
print("Public URL:", url)